In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.DataFrame(np.random.randint(0, 5, size=(2500, 3)), columns=list('XYZ'))
data['sum'] = data.sum(axis=1)
data.head()

,X,Y,Z,sum
0,4,4,2,10
1,2,1,3,6
2,0,3,1,4
3,3,2,3,8
4,4,0,3,7


# pcalg
## G2検定

In [23]:
import pcalg
import networkx as nx
import numpy as np
from gsq.ci_tests import ci_test_dis
from gsq.gsq_testdata import dis_data

dm = np.array(dis_data).reshape((10000, 5))
(g, sep_set) = pcalg.estimate_skeleton(indep_test_func=ci_test_dis,
                                 data_matrix=dm,
                                 alpha=0.01,
                                 levels=[3,2,3,4,2])
g = pcalg.estimate_cpdag(skel_graph=g, sep_set=sep_set)
g_answer = nx.DiGraph()
g_answer.add_nodes_from([0, 1, 2, 3, 4])
g_answer.add_edges_from([(0, 2), (1, 2), (1, 3), (4, 3)])
print('Edges are:', g.edges(), end='')
if nx.is_isomorphic(g, g_answer):
    print(' => GOOD')
else:
    print(' => WRONG')
    print('True edges should be:', g_answer.edges())

Edges are: [(0, 2), (1, 2), (1, 3), (4, 3)] => GOOD


## Fisher-Z検定

In [26]:
from citestfz.ci_tests import ci_test_gauss

In [28]:
cm = np.corrcoef(dm.T)

(g, sep_set) = pcalg.estimate_skeleton(indep_test_func=ci_test_gauss,
                                 data_matrix=dm,
                                 alpha=0.01,
                                 corr_matrix=cm)
g = pcalg.estimate_cpdag(skel_graph=g, sep_set=sep_set)
g_answer = nx.DiGraph()
g_answer.add_nodes_from([0, 1, 2, 3, 4])
g_answer.add_edges_from([(0, 2), (1, 2), (1, 3), (4, 3)])
print('Edges are:', g.edges(), end='')
if nx.is_isomorphic(g, g_answer):
    print(' => GOOD')
else:
    print(' => WRONG')
    print('True edges should be:', g_answer.edges())

Edges are: [(0, 2), (1, 2), (1, 3), (4, 3)] => GOOD
